# 8.1 即時台股看板

最後我們就發揮我們在過去所學，利用 **Excel、Python**、以及股價爬蟲來打造一個台股即時看板： 

完成品影片：https://youtu.be/5cf-G2IRVdk

![](https://drive.google.com/uc?export=download&id=1K8MqMzg-71bk33rB-qVSGSmn8PxCAaCP)

**備注：本教材由張佑成編寫，版權所有，翻印必究**


# 8.2 目標

該即時看板應該具備以下功能：

1. 顯示即時股價與漲跌幅度

![](https://drive.google.com/uc?export=download&id=161WgYDildws9AJMNaT8Oha0VdJXmYUy_)

2. 允許使用者輸入不同的股票代號
3. 程式會根據使用者輸入的股票代號顯示相對應的即時股價
4. 根據條件通知使用者的功能

# 8.3 顯示即時的股價

首先，我們先開啓 **PyXL台股看板.xlsx** 檔案

```python
import xlwings as xw

wb = xw.Book("PyXL台股看板.xlsx")
# 將 "觀察清單" 工作表存入 watch_list
watch_list = wb.sheets["觀察清單"]
```


# 8.3.1 截取即時股價資訊

回顧一下之前的教材，若要一次將多筆資料寫入到一個工作表的範圍内，我們可以先將所有資料都封裝到一個串列内

我們可以先觀察需要寫入的範圍的大小：

![](https://drive.google.com/uc?export=download&id=14V9m9pqMuAbcTfkdKWuuCSTSRku5IFRF)

如上圖顯示的，**B2:G2** 是一個 1 x 6 大小的範圍

在這樣的狀況下，我們必須透過 Python 把一個個股的資料轉變成以下格式：

```python
['玉山金', 27.15, 27.25, 27.15, 27.25, 27.05]
```

至於我們要如何取得股價資訊呢？我們在之前的課程打造的台股爬蟲並不適用於此，原因在於[聚財網](https://stock.wearn.com/cdata.asp?kind=2330)上的資訊並非即時資訊。

因此，我們需要一個會顯示即時股價資訊的網頁來當作資料來源，而在這一堂課，我們會使用[Yahoo!股市](https://tw.stock.yahoo.com/quote/2330.TW)：

```python
import requests
from bs4 import BeautifulSoup as BS

# 截取臺積電個股基本資訊
res = requests.get(f"https://tw.stock.yahoo.com/quote/2330.TW")
html = BS(res.text, "html.parser")
```

我們可以透過以下程式碼截取股票中文簡稱：

```python
h1 = html.findAll("h1", { "class": "C($c-link-text) Fw(b) Fz(24px) Mend(8px)"})[0]
name = h1.text
print(name)
```



截取頁面上即時的交易資訊：

```python
spans = html.findAll("span", {"class":["Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-down)", 
                                      "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)",
                                      "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c)"]})
```

將所有的 `span` 内的文字讀取出來：

```python
data = [float(span.text) for span in spans]
print(data)
```

檢查我們的資料：

```python
price_close = data[0]
price_open = data[1]
price_high = data[2]
price_low = data[3]
last_close = data[5]

print(f"2330 交易資訊：{name, price_open, price_high, price_low, price_close, last_close}")
```

將資料整理成我們需要的格式：

```python
print([name, data[1], data[2], data[3], data[0], data[5]])
```

# 8.3.2 將我們剛才處理資料的流程封裝到函數内

接下來修改網址最後的股票代號之後，我們的程式就可以截取多支不同的股票資訊了。
爲了保持後續程式碼的簡潔，我們可以將爬蟲封裝成一個函數：

```python
import requests
from bs4 import BeautifulSoup as BS

# 截取玉山銀個股基本資訊
# 輸入股票代號，回傳該股票的收盤價
def tw_stock_crawler(sid):
    res = requests.get(f"https://tw.stock.yahoo.com/quote/{sid}.TW")
    html = BS(res.text, "html.parser")

    spans = html.findAll("span", {"class":["Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-down)", 
                                          "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)",
                                          "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c)"]})
    h1 = html.findAll("h1", { "class": "C($c-link-text) Fw(b) Fz(24px) Mend(8px)"})[0]
    
    data = [float(span.text) for span in spans]
    #print(data)
    # 以字典形式回傳我們的資料
    return {
        "name": h1.text,
        "open": data[1],
        "high": data[2],
        "low": data[3],
        "close": data[0],
        "last": data[5]
    }
    
```

*若截取 Yahoo! 網頁失敗，請改使用聚財網：[程式碼](https://gist.github.com/yuyueugene84/a492a0ac809afcb9d49f738e130675ee)

測試我們寫好的函數：

```python
print(tw_stock_crawler("2412"))
```

將資料整理成我們要的格式：


```python
sid = "2884"

data = tw_stock_crawler(sid)

watch_list.range("B2").value = [
    data["name"],
    data["open"],
    data["high"],
    data["low"],
    data["close"],
    data["last"],
    data["close"]/data["last"]-1
]
```



我們就可以成功將資料寫入工作表了：

![](https://drive.google.com/uc?export=download&id=161WgYDildws9AJMNaT8Oha0VdJXmYUy_)

最後只需要輸入一個公式，我們就可以將漲跌幅度計算出來：

![](https://drive.google.com/uc?export=download&id=18mW3ehmeHvaW44QFWFxyYxR10VecXf6J)

# 8.4 動態截取使用者輸入的觀察清單

若要頻繁的修改程式碼來截取不同的股價資料，並不是一個聰明的做法，**因爲頻繁的修改程式碼會增加程式碼出錯的風險**

所以我們就利用 Excel 來當成一個簡易的前端界面，把使用程式的流程改成：

1. 動態截取使用者輸入的股票代號
2. 動態截取資料
3. 寫入工作表


我們先來看下面這段程式碼：

```python
# 查詢最後一列的列數
last_row = watch_list.range("A1").end("down").row

# 迭代每一列，取出 A 欄的股票代號
for i in range(2, last_row+1):
    stock_id = watch_list.range(f"A{i}").value
    print(stock_id)
    
# 2884
# 2330.0
# 2317.0
# 0050
```

可以發現 Python 會自動將我們填入的數字**截取成浮點數**，這樣會破壞爬蟲的運作

因此，我們需要修改一下程式碼，先偵測 A 欄資料的資料型別，再將其轉爲浮點數

```python
stock_id = watch_list.range(f"A{i}").value
# 處理字串
if type(stock_id) == float:
    stock_id = str(int(stock_id))
```

而上述這段程式碼可以被簡化成以下：

```python
stock_id = str(int(stock_id)) if type(stock_id) == float else str(stock_id)
```

修改一下我們原本的 for loop：

```python
# 偵測最後一行
last_row = watch_list.range("A1").end("down").row
# 從第二行到最後一行
for i in range(2, last_row+1):
    # 截取該行 A 欄的資料
    sid = watch_list.range(f"A{i}").value
    # 處理字串
    sid = str(int(sid)) if type(sid) == float else str(sid)
    print(sid)
    data = tw_stock_crawler(sid)
    print(data)
```

最後再加上寫入的動作：

```python
# 偵測最後一行
last_row = watch_list.range("A1").end("down").row
print(last_row)
# 從第二行到最後一行
for i in range(2, last_row+1):
    # 截取該行 A 欄的資料
    sid = watch_list.range(f"A{i}").value
    sid = str(int(sid)) if type(sid) == float else str(sid)
    print(sid)
    data = tw_stock_crawler(sid)
    print(data)
    # 將結果寫入觀察清單的同一行
    watch_list.range(f"B{i}").value = [
        data["name"],
        data["open"],
        data["high"],
        data["low"],
        data["close"],
        data["last"],
        data["close"]/data["last"]-1
    ]
```

最後我們的觀察清單就會顯示：

![](https://drive.google.com/uc?export=download&id=1clkVXuWuj9tOWk8XXCpogJRV7_q8d6Z4)

# 8.5 顯示時間

使用者很難顯示在看板上的資料判斷是否是即時的資訊，因此，我們可以加上一個顯示最後更新時間的功能：

![](https://drive.google.com/uc?export=download&id=1K8MqMzg-71bk33rB-qVSGSmn8PxCAaCP)

要打造出這樣的功能，我們可以再次把 Python 的 `strftime()` 拿出來使用：

```python
import time
time.strftime("%Y%m%d %H:%M:%S")
````

最後我們只需要指定被寫入時間的儲存格即可，做法很簡單，只需要先定義好顯示儲存格的名稱為 **上次更新**:
![](https://drive.google.com/uc?export=download&id=1B8H358y9yJVBLhHuzMsmSUyTOA41L4t0)

接下來將方才產生的時間寫入儲存格即可：

```python
import time

watch_list.range("上次更新").value = time.strftime("%Y%m%d %H:%M:%S")
```

大公告成！

# 8.6 加入 Line Notify 功能

我們希望看板能夠有通知使用者的功能，在遇到特殊狀況時，會通過 Line Notify 即時通報訊息給使用者：

```python
import requests

# 只需要把 content 的內容改成你需要傳遞的訊息即可
content = "你好,我是來自 Python 的訊息，啾咪！"

# Line Notify
line_url = "https://notify-api.line.me/api/notify"
token = "你申請到的 Line 權杖"
headers = {
        "Authorization": "Bearer " + token
    }

payload = {'message': content}
r = requests.post(line_url, headers = headers, params = payload)

```

# 8.6.1 將 Line Notify 封裝成函數

```python
def line_notify(msg, token):
    line_url = "https://notify-api.line.me/api/notify"
    headers = {
            "Authorization": "Bearer " + token
        }

    payload = {'message': msg}
    r = requests.post(line_url, headers = headers, params = payload)
    return
```

測試一下我們的函數：

```python
# 從工作表上使用者填入的 line token
token = watch_list.range("lineToken").value
# 發送 Line 訊息
line_notify("hello world!", token)
```

最後，我們可以根據條件，判斷是否要發送 Line 訊息。

舉例來説，如果要判斷某個股是否開始下跌：

```python
# 個股列數
i = 2
if watch_list.range(f"H{i}").value < 0:
    # 產生訊息
    msg = f"{data['name']} 目前開始下跌，請注意！"
    # 發送訊息
    line_notify(msg, token)
```

# 8.7 完整版程式碼

In [7]:
import time
import requests
import xlwings as xw
from bs4 import BeautifulSoup as BS

def tw_stock_crawler(sid):
    res = requests.get(f"https://tw.stock.yahoo.com/quote/{sid}.TW")
    html = BS(res.text, "html.parser")
    spans = html.findAll("span", {"class":["Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-down)", 
                                          "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c) C($c-trend-up)",
                                          "Fw(600) Fz(16px)--mobile Fz(14px) D(f) Ai(c)"]})
    h1 = html.findAll("h1", { "class": "C($c-link-text) Fw(b) Fz(24px) Mend(8px)"})[0]
    data = [float(span.text) for span in spans]
    # 以字典形式回傳我們的資料
    return {
        "name": h1.text,
        "open": data[1],
        "high": data[2],
        "low": data[3],
        "close": data[0],
        "last": data[5]
    }

def line_notify(msg, token):
    line_url = "https://notify-api.line.me/api/notify"
    headers = {
            "Authorization": "Bearer " + token
        }
    payload = {'message': msg}
    r = requests.post(line_url, headers = headers, params = payload)
    return

wb = xw.Book("PyXL台股看板.xlsx")
watch_list = wb.sheets["觀察清單"]

while True:
    # 偵測最後一行
    last_row = watch_list.range("A1").end("down").row
    print(last_row)
    # 從第二行到最後一行
    for i in range(2, last_row+1):
        # 截取該行 A 欄的資料
        sid = watch_list.range(f"A{i}").value
        sid = str(int(sid)) if type(sid) == float else str(sid)
        print(sid)
        data = tw_stock_crawler(sid)
        print(data)
        # 將結果寫入觀察清單的同一行
        watch_list.range(f"B{i}").value = [
            data["name"],
            data["open"],
            data["high"],
            data["low"],
            data["close"],
            data["last"],
            data["close"]/data["last"]-1
        ]
        
 
    # 記錄時間
    watch_list.range("上次更新").value = time.strftime("%Y%m%d %H:%M:%S")
    # 間隔 15 秒鐘，再爬一次資料
    time.sleep(5)

6
2330
{'name': '台積電', 'open': 581.0, 'high': 582.0, 'low': 579.0, 'close': 582.0, 'last': 574.0}
2454
{'name': '聯發科', 'open': 961.0, 'high': 969.0, 'low': 953.0, 'close': 966.0, 'last': 955.0}
0050
{'name': '元大台灣50', 'open': 132.65, 'high': 133.45, 'low': 132.5, 'close': 133.15, 'last': 131.55}
2303
{'name': '聯電', 'open': 50.4, 'high': 50.5, 'low': 50.0, 'close': 50.4, 'last': 50.2}
2603


IndexError: list index out of range

## 8.8 小結





在這一個教學，我們利用 Excel、Python、Line Notify、以及股價爬蟲打造出了一個台股即時看板。

在實作的過程當中，我們學會了如何將 **Excel 應用程式當作前端界面，讓 Python 當作後端程式**，根據這兩個工具的優勢進行整合，打造出了一個非常實用的解決方案。

接下來各位可以根據同樣的流程：
1. 打造個人或公司内部的後台資料的即時看板，請**技術人員開放 API，讓 Python 將即時的數據從後臺資料庫讀取出來，以自訂的格式呈現在 Excel 工作表上。**
2. 若希望能夠驗證一個想法，或是需要快速利用程式實作出一個 App 或網站的 Proto Type（雛形），但是卻沒有過多時間在刻畫前端界面時，不妨可以考慮將**期望的界面先在 Excel 上實作出來，若想法驗證成功，日後在實際開發時，也方便以此工作表與開發人員進行溝通。**

備注：

如果有同學對取得即時的台股資料有興趣，我在這邊推薦 [**fugle 即時股價 API**](https://developer.fugle.tw/docs/trading/intro)，開發者只需要[**花五分鐘開戶**](https://openaccount.fugle.tw/?referral=f-a7fbad3&utm_source=referral&utm_medium=link)，即可免費使用。